In [40]:
from bs4 import BeautifulSoup
import requests
import os
from urllib.request import urlretrieve
import regex as re

In [87]:
pattern_https = r'https://'
pattern_ebook = r'/ebooks/'
dl_links = set()

for page in pages:
    
    response = requests.get(url_pageless+str(page))
    html_file = BeautifulSoup(response.text, "html.parser")
    links = html_file.find_all("a")

    for link in links:
        try:
            slink = str(link['href'])
            match_https = re.match(pattern_https, slink)
            match_ebook = re.match(pattern_ebook, slink)
            if not match_https and match_ebook:
                dl_links.add("https://standardebooks.org" + link["href"] + "/download?format=epub")
        except KeyError:
            continue

In [88]:
dl_links

{'https://standardebooks.org/ebooks/a-a-milne/the-red-house-mystery/download?format=epub',
 'https://standardebooks.org/ebooks/a-a-milne/winnie-the-pooh/download?format=epub',
 'https://standardebooks.org/ebooks/a-merritt/the-moon-pool/download?format=epub',
 'https://standardebooks.org/ebooks/a-w-tozer/the-pursuit-of-god/download?format=epub',
 'https://standardebooks.org/ebooks/adam-mickiewicz/pan-tadeusz/maude-ashurst-biggs/download?format=epub',
 'https://standardebooks.org/ebooks/adam-smith/the-wealth-of-nations/download?format=epub',
 'https://standardebooks.org/ebooks/aesop/fables/v-s-vernon-jones/download?format=epub',
 'https://standardebooks.org/ebooks/agatha-christie/poirot-investigates/download?format=epub',
 'https://standardebooks.org/ebooks/agatha-christie/the-big-four/download?format=epub',
 'https://standardebooks.org/ebooks/agatha-christie/the-man-in-the-brown-suit/download?format=epub',
 'https://standardebooks.org/ebooks/agatha-christie/the-murder-of-roger-ackroyd/d

In [90]:
for index, dl_link in enumerate(dl_links):
    response_dl_page = requests.get(dl_link)
    dl_page = BeautifulSoup(response_dl_page.content, 'html.parser')
    meta_refresh_tag = dl_page.find('meta', attrs={'http-equiv': 'refresh'})['content']
    real_dl_link = meta_refresh_tag.split('; url=')[1]
    epub_file = requests.get("https://standardebooks.org" + real_dl_link)

    iindex = index + 1
    filename = "ebook" + str(iindex) + ".epub"
    with open(filename, mode="wb") as f:
        f.write(epub_file.content)